# Importación de módulos

In [1]:
import os
import pandas as pd

# Lectura de la Base de Datos
Para descargar la base de datos original se necesita ir a la siguiente [ruta](https://sio.cnsf.gob.mx/)

## Definición de ruta

In [2]:
read_dir = r'C:\Users\ed_22\Documents\SIO'
read_file = 'ER_sep2023.xlsx'
read_path = os.path.join(read_dir, read_file)

## Variables globales

In [18]:
FIELDS = {
    'FECHA_CORTE':'corte',
    'NOMBRE_CORTO':'nombre',
    'DESCRIPCION':'cuenta',
    'DESC_OPERACION':'ramo',
    'IMPORTE':'importe'
}

In [19]:
ER = pd.read_excel(read_path, usecols=FIELDS.keys()).rename(columns=FIELDS)

In [20]:
ER.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518811 entries, 0 to 518810
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   corte    518811 non-null  datetime64[ns]
 1   nombre   518811 non-null  object        
 2   cuenta   518811 non-null  object        
 3   ramo     518811 non-null  object        
 4   importe  518811 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 19.8+ MB


In [21]:
ER.nombre.unique()

array(['Crédito Afianzador', 'Fianzas Guardiana Inbursa',
       'Liberty Fianzas', 'Fianzas Asecam', 'Afianzadora CBL Fiducia',
       'Mapfre Fianzas', 'CESCE Fianzas México', 'Zurich Fianzas México',
       'MBIA México', 'Plan Seguro', 'Medi Access Seguros de Salud',
       'BBVA Bancomer Seguros Salud', 'AXA Salud', 'General de Salud',
       'Servicios Integrales de Salud Nova', 'Seguros Centauro',
       'Dentegra Seguros Dentales', 'Odontored Seguros Dentales',
       'Pensiones Sura', 'Pensiones Banorte', 'HSBC Pensiones',
       'Pensiones BBVA Bancomer', 'Profuturo GNP Pensiones',
       'Principal Pensiones', 'Pensiones Banamex',
       'Metlife Pensiones México', 'Pensiones Inbursa', 'Seguros Banorte',
       'Allianz México', 'Patrimonial Inbursa',
       'Atradius Seguros de Crédito', 'Seguros El Potosí',
       'General de Seguros', 'Seguros Sura', 'AIG Seguros México',
       'La Latinoamericana Seguros', 'Seguros Ve por Más',
       'Zurich Santander Seguros México', 

In [22]:
ER.ramo.unique()

array(['Total', '*Fianzas', '**Fidelidad', '***Individual',
       '***Colectivo', '**Judicial o Penal', '***Penales',
       '***No penales', '***Amparan conductores de automóviles',
       '**Administrativo', '***Por Obra', '***Proveeduría', '***Fiscales',
       '***Arrendamiento', '***Otras de Administrativo', '**De crédito',
       '***Suministro', '***Compra Venta', '***Otras de Crédito',
       '***Financiera', '*Da?os', '**Garantía financiera',
       '*Accidentes y Enfermedades', '***Gastos Médicos', '***Salud',
       '***Accidentes Personales', '*Pensiones',
       '***IMSS Incapacidad Permanente RT',
       '***IMSS Muerte (riesgos de trabajo)',
       '***IMSS Invalidez (invalidez y vida)',
       '***IMSS Muerte (invalidez y vida)',
       '***IMSS Jubilación (retiro, cesantía y vejez)',
       '***ISSSTE Incapacidad Permanente RT',
       '***ISSSTE Muerte (riesgos de trabajo)',
       '***ISSSTE Invalidez (invalidez y vida)',
       '***ISSSTE Muerte (invalidez y vida)'

In [23]:
'***Administrativo'.count('*')

3

In [24]:
def asignar_nivel(cadena):
    """ devuelve el nivel correspondiente al ramo""" 
    return cadena.count('*')

In [26]:
ER['nivel'] = ER.ramo.apply(asignar_nivel)

In [27]:
ER.head()

,corte,nombre,cuenta,ramo,importe,nivel
0,2016-03-31,Crédito Afianzador,Directo,Total,17.68,0
1,2016-06-30,Crédito Afianzador,Directo,Total,19.30,0
2,2016-09-30,Crédito Afianzador,Directo,Total,21.39,0
3,2016-12-31,Crédito Afianzador,Directo,Total,24.54,0
4,2017-03-31,Crédito Afianzador,Directo,Total,17.77,0


In [10]:
ER.cuenta.unique()

array(['Directo', 'Primas Cedidas', 'Primas De Retención',
       'Primas de Retención Devengadas', 'Costo Neto de Adquisición',
       'Comisiones a Agentes',
       'Costo Neto de Siniestralidad, Reclamaciones y Otras Obligaciones Pendientes de Cumplir',
       'Utilidad (Pérdida) Técnica', 'Gastos de Operación Netos',
       'Resultado Integral de Financiamiento',
       'Utilidad (Pérdida) antes de Impuestos a la Utilidad',
       'Utilidad  (Pérdida) del Ejercicio',
       'Utilidad (Pérdida) de la Operación', 'Tomado',
       'Compensaciones Adicionales a Agentes'], dtype=object)

In [30]:
ER.query('cuenta == "Directo"').importe.sum() + ER.query('cuenta == "Tomado"').importe.sum() - ER.query('cuenta == "Primas Cedidas"').importe.sum()

30994940.24000001

In [29]:
ER.query('cuenta == "Primas De Retención"').importe.sum()

30986033.189999998

In [11]:
df = ER.copy()

In [12]:
df['nombre'] = df.nombre.str.replace('?', 'ñ', regex=True)
df['desc'] = df.desc.str.replace('?', 'ñ', regex=True)
df['desc_op'] = df.desc_op.str.replace('*', '', regex=True)
df['desc_op'] = df.desc_op.str.replace('?', 'ñ', regex=True)

In [13]:
df.pivot_table(columns=['corte'], index=['nombre'], values='importe').head()

corte,2016-03-31,2016-06-30,2016-09-30,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,...,2021-06-30,2021-09-30,2021-12-31,2022-03-31,2022-06-30,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30
nombre,,,,,,,,,,,,,,,,,,,,,
Armour Secure Insurance,3.601818,12.290000,19.134545,25.320000,5.489091,12.890000,20.369091,27.010909,2.924545,12.928182,...,16.433333,20.963333,27.811667,2.730909,18.486667,25.205833,33.028333,6.253333,16.636667,35.802500
A.N.A. Compañía de Seguros,41.121209,84.565824,124.839890,183.015385,62.841868,119.588791,178.265604,248.655275,78.953407,150.985385,...,213.140612,320.915306,462.012755,121.909694,234.652653,340.947959,479.295204,119.757959,238.570612,354.420612
AIG Seguros México,48.982303,62.695184,91.103323,139.053180,44.170154,83.512215,136.858532,193.711713,37.872190,101.395683,...,85.921780,125.647901,168.820641,44.229822,90.552997,140.481357,178.383569,74.707713,114.218710,164.408271
AXA Salud,20.896000,35.973000,46.673636,60.734545,23.023000,30.050000,44.248000,57.913000,12.832000,28.049000,...,13.520833,18.150833,19.870833,78.015000,81.685833,102.615833,87.980000,78.431667,79.917500,103.945000
AXA Seguros,321.767366,587.335182,835.292067,1121.232230,311.173842,552.388361,824.500820,1116.206307,304.428943,618.268905,...,752.010896,1071.704800,1413.980869,424.760571,806.328900,1171.188729,1554.810220,474.501931,881.061691,1279.866754


# Exploratory Data Analysis
También conocido como **EDA**. Nos permite hacer una exploración de los datos para darnos una idea de qué tipo de análisis podemos llegar a hacer.

## YData Profiling
Para ver la documentación oficial [aquí](https://docs.profiling.ydata.ai/latest/)

**Instalación**

    pip install ydata-profiling

In [6]:
!pip install ydata-profiling


     ---------------------------------------- 0.0/84.4 kB ? eta -:--:--
     ---------------------------------------- 84.4/84.4 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/357.9 kB ? eta -:--:--
   ---------------- ----------------------- 143.4/357.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 357.9/357.9 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/296.5 kB ? eta -:--:--
   ---------------------------------------- 296.5/296.5 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/102.7 kB ? eta -:--:--
   ---------------------------------------- 102.7/102.7 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/666.3 kB ? eta -:--:--
   ---------------------------------- ---- 583.7/666.3 kB 12.2 MB/s eta 0:00:01
   ---------------------------------------  665.6/666.3 kB 8.

In [14]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Profiling Report")

In [15]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Ejercicios
1. Obtener la prima directa del ramo Agrícola de las 5 principales instituciones.
2. ¿Qué institución cuenta con